In [21]:
import pandas as pd
import math as math
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply, TimeDistributed, Flatten
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.special import boxcox, inv_boxcox

import numpy as np
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit


In [22]:
data = pd.read_csv(r"C:\Users\patte\Documents\apps\covidLSTM\COVID-19\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_confirmed_global.csv");

In [23]:
def fullNaN(countries_data):
    result = []
    for country_data in countries_data:
        result.append(country_data.fillna('Total'))
    return result

def country_with_province_data_prep(countries_data):
    result = []
    for country_data in countries_data:
        country_name = country_data['Country/Region'].values[0]
        del country_data['Country/Region']
        del country_data['Lat'];
        del country_data['Long'];
        country_data = country_data.melt(id_vars=['Province/State'], var_name='Date', value_name='Confirmed Cases');
        country_data['Date'] = pd.to_datetime(country_data['Date']);
        country_data = country_data.pivot(index='Date', columns='Province/State', values='Confirmed Cases');
        country_total = pd.DataFrame()
        country_total[country_name + ' Total Confirmed Cases '] = country_data.sum(axis=1)
        result.append(country_total)
    return result

def get_training_data_no_provinces():
    korea_data = data.loc[data['Country/Region'] == 'Korea, South']
    italy_data = data.loc[data['Country/Region'] == 'Italy']
    spain_data = data.loc[data['Country/Region'] == 'Spain']
    germany_data = data.loc[data['Country/Region'] == 'Germany']

    return [korea_data, italy_data, spain_data, germany_data]

def get_training_data_with_provinces():
    france_data = data.loc[data['Country/Region'] == 'France']
    uk_data = data.loc[data['Country/Region'] == 'United Kingdom']
    china_data = data.loc[data['Country/Region'] == 'China']
    
    return [france_data, uk_data, china_data]

def country_no_province_data_prep(countries):
    result = []
    for country_data in countries:
        country_name = country_data['Country/Region'].values[0]
        del country_data['Lat']
        del country_data['Long']
        del country_data['Province/State']
        country_data = country_data.melt(id_vars=['Country/Region'], var_name='Date', value_name='Confirmed Cases')
        ## Data prep for Korea
        country_data['Date'] = pd.to_datetime(country_data['Date']);
        country_data = country_data.pivot(index='Date', columns='Country/Region', values='Confirmed Cases');
        country_total = pd.DataFrame()
        country_total[country_name + ' Total Confirmed Cases'] = country_data.sum(axis=1)
        result.append(country_total)
    return result

In [24]:
countries_provinces_data = get_training_data_with_provinces()
countries_data = get_training_data_no_provinces()
x1 = country_with_province_data_prep(countries_provinces_data)
x2 = country_no_province_data_prep(countries_data)
input_data = x1 + x2

In [25]:
## Standardization
def normalize_data(matrix):
    scaler_array = []
    normalized_vectors = []
    for vector in matrix:
        scaler = MinMaxScaler().fit(vector)
        normalized_vector = scaler.transform(vector)
        scaler_array.append(scaler)
        normalized_vectors.append(normalized_vector)
    return scaler, normalized_vectors

In [26]:
scaler, X = normalize_data(input_data)

In [28]:
## Config for model
Tx = X[3].shape[0]
Ty = Tx
x_dim = len(X)
lstm_x_hidden = 24
lstm_y_hidden = 32
x_dim

7

In [32]:
X = Input(shape=(Tx, x_dim))
output = LSTM(lstm_x_hidden, return_state=True)(X)

(None, 88, 7)


2

In [14]:
model = Model(inputs=[X])

TypeError: _base_init() got an unexpected keyword argument 'inputs'